Convolutional neural networks

**numpy** is used for numerial computing and Images can be represented as multi dimensional arrays which represents pixel value

In [ ]:
import numpy as np

**num_filters = 8** #passed as one argument

8,3,3 / 9 ---> 9 & 3,3 will be cancel balance 8 will get *self filters will be 8*

In [ ]:
class Conv3X3:

  def __init__(self,num_filters):
    self.num_filters = num_filters
    self.filters = np.random.randn(num_filters,3, 3)/9 #8,3,3


**VALID PADDING IS USED** it reduces the one row and column from the image
*h--28*
*w--28*

In [ ]:
class Conv3x3:

  def iterate_regions(self, image):

    h, w = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)] #0:2,0:2--->(0,1,2),(0,1,2)--->3,3
        yield im_region, i, j #yield is used for memory allocation every time it will not store (3,3),(4,4),(5,5)values it holds (3,3) as i,j

  def forward(self, input):

    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_filters)) #every row&col will be filled with zeros (26,26,8)

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.sum(im_region * self.filters, axis=(1, 2)) #crosscorrelation

    return output

In [ ]:
!pip install mnist


In [ ]:
import mnist
from conv import Conv3x3


train_images = mnist.train_images()
train_labels = mnist.train_labels()

conv = Conv3x3(8)
output = conv.forward(train_images[0])
print(output.shape) # (26, 26, 8)

(26, 26, 8)


**Pool size - 2**
 axis=(0, 1) ---> maximize over the first two dimension of height and width

In [ ]:
import numpy as np

class MaxPool2:


  def iterate_regions(self, image):

    h, w, _ = image.shape
    new_h = h // 2 #14
    new_w = w // 2 #14

    for i in range(new_h): #(0,14)
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)] #(0:2),(0:2)
        yield im_region, i, j

  def forward(self, input):

    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1)) #argmax is used for to find the max from

    return output

In [ ]:

import mnist
from conv import Conv3x3
from maxpool import MaxPool2


train_images = mnist.train_images()
train_labels = mnist.train_labels()

conv = Conv3x3(8)
pool = MaxPool2()

output = conv.forward(train_images[0])
output = pool.forward(output)
print(output.shape) # (13, 13, 8)

(13, 13, 8)


**Softmax** probability distribution which takes negative value and it mainly used in multi-class classification problems

*13 * 13 * 8 --> input_len* - 1352*


In [ ]:
import numpy as np

class Softmax:

  def __init__(self, input_len, nodes):

    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):

    input = input.flatten()

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases
    exp = np.exp(totals) #formula
    return exp / np.sum(exp, axis=0)

*from 1 argument to 8 filters * --># 28x28x1 -> 26x26x8

*from filter as 8 to maxpooling* --> # 26x26x8 -> 13x13x8

*from pooling to output* --># 13x13x8 -> 10

In [ ]:
import mnist
import numpy as np
from conv import Conv3x3
from maxpool import MaxPool2
from softmax import Softmax


test_images = mnist.test_images()[:1000]
test_labels = mnist.test_labels()[:1000]

conv = Conv3x3(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool2()                  # 26x26x8 -> 13x13x8
softmax = Softmax(13 * 13 * 8, 10) # 13x13x8 -> 10

def forward(image, label):

  out = conv.forward((image / 255) - 0.5) #normalize transform the image from [0, 255] to [-0.5, 0.5]
  out = pool.forward(out)
  out = softmax.forward(out)


  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0 #output value will be chcek through this condition Eg: if 8 is out then label should get 8 if not acc will be 0

  return out, loss, acc

print('MNIST CNN initialized!')

loss = 0
num_correct = 0
for i, (im, label) in enumerate(zip(test_images, test_labels)):

  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc


  if i % 100 == 99:
    print(
      '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
      (i + 1, loss / 100, num_correct)
    )
    loss = 0
    num_correct = 0

MNIST CNN initialized!
[Step 100] Past 100 steps: Average Loss 2.303 | Accuracy: 8%
[Step 200] Past 100 steps: Average Loss 2.302 | Accuracy: 14%
[Step 300] Past 100 steps: Average Loss 2.302 | Accuracy: 14%
[Step 400] Past 100 steps: Average Loss 2.302 | Accuracy: 16%
[Step 500] Past 100 steps: Average Loss 2.303 | Accuracy: 7%
[Step 600] Past 100 steps: Average Loss 2.303 | Accuracy: 10%
[Step 700] Past 100 steps: Average Loss 2.303 | Accuracy: 12%
[Step 800] Past 100 steps: Average Loss 2.302 | Accuracy: 15%
[Step 900] Past 100 steps: Average Loss 2.303 | Accuracy: 9%
[Step 1000] Past 100 steps: Average Loss 2.302 | Accuracy: 13%
